<a href="https://colab.research.google.com/github/Praveen76/Introduction-to-RAY/blob/main/Introduction_to_Ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Learning Objectives

At the end of the experiment, you will be able to:

* load the data into RayDMatrix
* train the XGBoost Ray model and save it
* tune the Hyperparameters using Ray tune

## Introduction

Compute demands for machine learning (ML) training have grown 10x every
18 months since 2010. Over the same time period, the compute capabilities of
AI accelerators such as GPUs and TPUs have less than doubled. This means
that every year and a half organizations need 5x more AI accelerators/nodes
to train the latest ML models and leverage cutting edge ML capabilities.
Distributed computing is the only way to meet these requirements.

While solutions such as AWS SageMaker and GCP Vertex AI have emerged
to help organizations deal with scaling AI workloads, these solutions put
significant constraints on how applications are developed and which libraries
they can use. This makes it difficult to keep up with the latest models and
algorithms, and freely integrate with the rapidly evolving open ML ecosystem.

Ray, addresses these challenges head on by
allowing ML engineers and developers to scale their workloads effortlessly
from their laptops to the cloud without the need to build complex compute
infrastructures.

### <img src='https://global.discourse-cdn.com/business7/uploads/ray/original/1X/8f4dcb72f7cd34e2a332d548bd65860994bc8ff1.png' width=20px> **Ray**

Ray is an open-source unified framework for scaling AI and Python applications like machine learning. It provides the compute layer for parallel processing and reduce the need of a distributed systems expert. Ray minimizes the complexity of running your distributed individual and end-to-end machine learning workflows with these components:

- Scalable libraries for common machine learning tasks such as data preprocessing, distributed training, hyperparameter tuning, reinforcement learning, and model serving.

- Pythonic distributed computing primitives for parallelizing and scaling Python applications.

- Integrations and utilities for integrating and deploying a Ray cluster with existing tools and infrastructure such as Kubernetes, AWS, GCP, and Azure.
<br>

Some common ML workloads that individuals, organizations, and companies leverage Ray to build their AI applications include:

- Batch inference on CPUs and GPUs
- Parallel training
- Model serving
- Distributed training of large models
- Parallel hyperparameter tuning experiments
- Reinforcement learning
- ML platform


### **Ray framework**

<center>
<img src="https://datascienceimages.s3.eu-north-1.amazonaws.com/MLOps/Introduction_to_Ray/Ray_framework.png" width=500px></center>
<br><br>

Ray's unified compute framework consists of three layers:

- ***Ray AI Libraries:*** An open-source, Python, domain-specific set of libraries that equip ML engineers, data scientists, and researchers with a scalable and unified toolkit for ML applications.

- ***Ray Core:*** An open-source, Python, general purpose, distributed computing library that enables ML engineers and Python developers to scale Python applications and accelerate machine learning workloads.

- ***Ray Clusters:*** A set of worker nodes connected to a common Ray head node. Ray clusters can be fixed-size, or they can autoscale up and down according to the resources requested by applications running on the cluster.

<br>

Each of Ray's five native libraries distributes a specific ML task:

- **`Data`**: Scalable, framework-agnostic data loading and transformation across training, tuning, and prediction

- **`Train`**: Distributed multi-node and multi-core model training with fault tolerance that integrates with popular training libraries

- **`Tune`**: Scalable hyperparameter tuning to optimize model performance

- **`Serve`**: Scalable and programmable serving to deploy models for online inference, with optional microbatching to improve performance

- **`RLlib`**: Scalable distributed reinforcement learning workloads


Find the official Ray website [here](https://www.ray.io/), and its documentation [here](https://docs.ray.io/en/latest/ray-overview/index.html).

### Setup Steps:

### Install necessary packages

In [ ]:
!pip -q install ray
!pip -q install ray[tune]
!pip -q install xgboost_ray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.4/138.4 kB 2.7 MB/s eta 0:00:00


### Import necessary libraries

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

import xgboost as xgb
from xgboost_ray import RayDMatrix, RayParams, train, predict

from ray import tune
from ray import train as raytrain

### Load the data

In [ ]:
train_x, train_y = load_breast_cancer(return_X_y=True)
train_x.shape, train_y.shape

((569, 30), (569,))

In [ ]:
train_x

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [ ]:
train_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

### XGBoost-Ray uses the same API as core XGBoost

There are only two differences:

* Instead of using a `xgboost.DMatrix`, it uses `xgboost_ray.RayDMatrix` object

* There is an additional `ray_params` parameter that is used to configure distributed training (it takes a `RayParams` object)

**Data loading**

Data is passed to XGBoost-Ray via a `RayDMatrix` object.

The `RayDMatrix` lazy loads data and stores it sharded in the Ray object store. The Ray XGBoost actors then access these shards to run their training on.

A `RayDMatrix` support various data and file types, like Pandas DataFrames, Numpy Arrays, CSV files and Parquet files.

In [ ]:
train_set = RayDMatrix(train_x, train_y)

In [ ]:
ray_params = RayParams(num_actors = 2,               # Number of remote actors
                       cpus_per_actor = 1
                       )

### Train the XGBoost Ray model and save it

In [ ]:
evals_result = {}
bst = train(
    params={
        "objective": "binary:logistic",                # tells XGBoost that we aim to train a logistic regression model for a binary classification task
        "eval_metric": ["logloss", "error"],
    },
    dtrain=train_set,
    evals_result=evals_result,
    evals=[(train_set, "train")],
    verbose_eval=False,
    ray_params=ray_params)

bst.save_model("model.xgb")

2024-06-17 18:43:19,279	INFO worker.py:1753 -- Started a local Ray instance.
2024-06-17 18:43:26,148	INFO main.py:1140 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
2024-06-17 18:43:43,459	INFO main.py:1191 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=886) [18:43:43] task [xgboost.ray]:136456321431104 got new rank 0
2024-06-17 18:43:46,942	INFO main.py:1708 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 24.00 seconds (3.47 pure XGBoost training time).
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:43:47] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


### Final training error

In [ ]:
print("Final training error: {:.4f}".format(evals_result["train"]["error"][-1]))
print("Final training accuracy: {:.4f}".format(1 - evals_result["train"]["error"][-1]))

Final training error: 0.0053
Final training accuracy: 0.9947


### Prediction

Here, we will create an object of regular non-distributed API instance i.e. `xgboost.Booster`, and pass the saved XGBoost-Ray model.

In [ ]:
dpred = RayDMatrix(train_x, train_y)

bst = xgb.Booster(model_file="model.xgb")                    # non-distributed XGBoost API instance

pred_ray = predict(bst,
                   dpred,
                   ray_params = RayParams(num_actors=2)      # The data will be split across two actors. The result array will integrate this data in the correct order.
                   )

print(pred_ray)

2024-06-17 18:43:47,244	INFO main.py:1758 -- [RayXGBoost] Created 2 remote actors.
2024-06-17 18:43:53,807	INFO main.py:1775 -- [RayXGBoost] Starting XGBoost prediction.


[0.09144595 0.05673993 0.03008196 0.10851309 0.09144595 0.1226117
 0.03008196 0.03145875 0.03795947 0.09715987 0.11452682 0.03008196
 0.03008196 0.04623247 0.06943818 0.03008196 0.03738927 0.03008196
 0.03008196 0.97716796 0.98008084 0.98008084 0.0675434  0.03008196
 0.03008196 0.03620729 0.03974995 0.03008196 0.03008196 0.09527509
 0.03008196 0.03008196 0.03008196 0.03008196 0.03008196 0.03008196
 0.04384386 0.95125544 0.26743954 0.05803299 0.5169782  0.30979052
 0.03008196 0.03145875 0.11796542 0.03620729 0.98008084 0.04324226
 0.98008084 0.93069357 0.9776062  0.9790822  0.9698948  0.03261407
 0.05082424 0.9698948  0.03008196 0.03145875 0.9790822  0.9698948
 0.9731866  0.98008084 0.03008196 0.98008084 0.03145875 0.03145875
 0.9700466  0.9698948  0.86911684 0.9698948  0.03008196 0.98008084
 0.03008196 0.26402172 0.98008084 0.03008196 0.9691353  0.05734929
 0.03958243 0.98008084 0.9700466  0.8453193  0.03008196 0.03008196
 0.98008084 0.03008196 0.14372692 0.03008196 0.9427549  0.922841

In [ ]:
# Convert model output to labels
prediction = [int(i > 0.5) for i in pred_ray]
print(prediction)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(prediction, train_y)

0.9947275922671354

## Hyperparameter Tuning with Ray Tune

By using tuning libraries such as **Ray Tune** we can try out combinations of hyperparameters. Using sophisticated search strategies, these parameters can be selected so that they are likely to lead to good results (avoiding an expensive exhaustive search).

Also, trials that do not perform well can be preemptively stopped to reduce waste of computing resources. Ray Tune also takes care of training these runs in parallel, greatly increasing search speed.

**Steps:**
1. Put the non-distributed XGBoost training call into a function accepting parameter configurations (`train_breast_cancer_model()` in the example below)

2. Define the parameter search space (`config` dictionary)

3. Create `tune.Tuner()` object:
    * pass training call function
    * pass tuning configuration `tune.TuneConfig()`
        * `num_samples`: number of different hyperparameter configurations from the search space
        * `metric`: the metric to optimized
        * `mode`: should either be min or max, depending on whether the metric is to be minimized or maximized
    * pass parameter search space

4. Call `tuner.fit()`

In [ ]:
# Function for XGBoost training
def train_breast_cancer_model(config):
    # Load dataset
    data, labels = load_breast_cancer(return_X_y=True)
    # Split into train and test set
    train_x, test_x, train_y, test_y = train_test_split(data, labels, test_size=0.25)

    # Build input matrices for XGBoost
    train_set = xgb.DMatrix(train_x, label=train_y)
    test_set = xgb.DMatrix(test_x, label=test_y)

    # Train the classifier
    results = {}
    xgb.train(
        params=config,
        dtrain=train_set,
        evals=[(test_set, "eval")],
        evals_result=results,
        verbose_eval=False,
    )
    # Return prediction accuracy
    accuracy = 1.0 - results["eval"]["error"][-1]
    raytrain.report({"mean_accuracy": accuracy, "done": True})        #  instead of returning the accuracy value, we report it back to Tune using session.report()


# Define the parameter search space
config = {
    "objective": "binary:logistic",                            # tells XGBoost that we aim to train a logistic regression model for a binary classification task
    "eval_metric": ["logloss", "error"],
    "max_depth": tune.randint(1, 9),                           # hyperparameter    'tune.randint(min, max)' chooses a random integer value between min and max
    "min_child_weight": tune.choice([1, 2, 3]),                # hyperparameter    'tune.choice([a, b, c])' chooses one of the items of the list at random
    "subsample": tune.uniform(0.5, 1.0),                       # hyperparameter    'tune.uniform(min, max)' samples a floating point number between min and max
    "eta": tune.loguniform(1e-4, 1e-1),                        # hyperparameter    'tune.loguniform(min, max, base=10)' samples a floating point number between min and max,
                                                               #                    but applies a logarithmic transformation to these boundaries first
    }

tuner = tune.Tuner(
    train_breast_cancer_model,
    tune_config = tune.TuneConfig(num_samples=10,              # sample 10 different hyperparameter configurations from the search space
                                  metric="mean_accuracy",      # the metric to optimized
                                  mode="max"                   # the mode should either be min or max, depending on whether the metric is to be minimized or maximized
                                  ),
    param_space = config                                       # parameter search space
)

results = tuner.fit()

+----------------------------------------------------------------------------------+
| Configuration for experiment     train_breast_cancer_model_2024-06-17_18-44-32   |
+----------------------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator                           |
| Scheduler                        FIFOScheduler                                   |
| Number of trials                 10                                              |
+----------------------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_breast_cancer_model_2024-06-17_18-44-32
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-06-17_18-43-14_331037_235/artifacts/2024-06-17_18-44-32/train_breast_cancer_model_2024-06-17_18-44-32/driver_artifacts`

Trial status: 10 PENDING
Current time: 2024-06-17 18:44:37. Total running time: 0s
Logical resourc

2024-06-17 18:45:10,542	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_breast_cancer_model_2024-06-17_18-44-32' in 0.0169s.



Trial train_breast_cancer_model_a305f_00008 completed after 1 iterations at 2024-06-17 18:45:10. Total running time: 33s
+----------------------------------------------------------------+
| Trial train_breast_cancer_model_a305f_00008 result             |
+----------------------------------------------------------------+
| checkpoint_dir_name                                            |
| time_this_iter_s                                       0.06148 |
| time_total_s                                           0.06148 |
| training_iteration                                           1 |
| mean_accuracy                                          0.62238 |
+----------------------------------------------------------------+

Trial status: 10 TERMINATED
Current time: 2024-06-17 18:45:10. Total running time: 33s
Logical resource usage: 1.0/2 CPUs, 0/0 GPUs
Current best trial: a305f_00001 with mean_accuracy=0.9370629370629371 and params={'objective': 'binary:logistic', 'eval_metric': ['logloss', '

In [ ]:
# Best hyperparameters
best_params = results.get_best_result().config
best_params

{'objective': 'binary:logistic',
 'eval_metric': ['logloss', 'error'],
 'max_depth': 3,
 'min_child_weight': 1,
 'subsample': 0.9264192917869138,
 'eta': 0.08734472028779346}

In [ ]:
# All trial results
df = results.get_dataframe()
df.head(2)

,mean_accuracy,done,timestamp,checkpoint_dir_name,training_iteration,trial_id,date,time_this_iter_s,time_total_s,pid,...,node_ip,time_since_restore,iterations_since_restore,config/objective,config/eval_metric,config/max_depth,config/min_child_weight,config/subsample,config/eta,logdir
0,0.615385,True,1718649884,None,1,a305f_00000,2024-06-17_18-44-44,0.091428,0.091428,1377,...,172.28.0.12,0.091428,1,binary:logistic,"[logloss, error]",4,2,0.647329,0.000546,a305f_00000
1,0.937063,True,1718649884,None,1,a305f_00001,2024-06-17_18-44-44,0.051082,0.051082,1378,...,172.28.0.12,0.051082,1,binary:logistic,"[logloss, error]",3,1,0.926419,0.087345,a305f_00001
